In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import pandas as pd
import numpy as np
from func_linear import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
x = pd.read_csv('carvan_train.csv')
y = x['V86']
df_test = pd.read_csv('carvan_test.csv')

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
df = pd.read_csv('L0.txt', delimiter='\t', skiprows=1)
l0 = {}

for i in range(0, 41):
    l0[i] = df['Label'][i]

l1 = {1: '20-30', 2: '30-40', 3: '40-50', 4: '50-60', 5: '60-70', 6: '70-80'}

l2 = {1: 'Successful hedonists', 2: 'Driven Growers', 3: 'Average Family', 4: 'Career Loners', 5: 'Living well', 6: 'Cruising Seniors', 
      7: 'Retired and Religeous', 8: 'Family with grown ups', 9: 'Conservative families', 10: 'Farmers'}

l3 = {0: '0', 1: '1 - 10', 2: '11 - 23', 3: '24 - 36', 4: '37 - 49', 5: '50 - 62', 6: '63 - 75', 7: '76 - 88', 8: '89 - 99', 9: '100'}

l4 = {0: '0', 1: '1-49', 2: '50-99', 3: '100-199', 4: '200-499', 5: '500-999', 6: '1000-4999', 7: '5000-9999', 8: '10000-19999', 9: '20000'}

In [7]:
def avg(d):
    app = []
    for i in d.values():
        k = i.split('-')
        app.append(k)
    new_value = []
    for i in app:
        if len(i) > 1:
            new_value.append((int(i[0].strip())+int(i[1].strip()))/2)
        else:
            new_value.append(int(i[0].strip()))
    return new_value

l_1 = avg(l1)
for i,j in zip(l1.keys(), l_1):
    l1[i] = j

l_3 = avg(l3)
for i,j in zip(l3.keys(), l_3):
    l3[i] = j

l_4 = avg(l4)
for i,j in zip(l4.keys(), l_4):
    l4[i] = j

In [9]:
pipe = Pipeline([
    ('dummy1', create_dummy('V1')),
    ('dummy2', create_dummy('V5')),
    ('custom1', custom('V4', l1)),
    ('custom2', custom('V6', l3)),
    ('custom3', custom('V44', l4))
])

In [11]:
pipe.fit(x)

Pipeline(steps=[('dummy1', create_dummy(var='V1')),
                ('dummy2', create_dummy(var='V5')),
                ('custom1',
                 custom(pairs={1: 25.0, 2: 35.0, 3: 45.0, 4: 55.0, 5: 65.0,
                               6: 75.0},
                        var='V4')),
                ('custom2',
                 custom(pairs={0: 0, 1: 5.5, 2: 17.0, 3: 30.0, 4: 43.0, 5: 56.0,
                               6: 69.0, 7: 82.0, 8: 94.0, 9: 100},
                        var='V6')),
                ('custom3',
                 custom(pairs={0: 0, 1: 25.0, 2: 74.5, 3: 149.5, 4: 349.5,
                               5: 749.5, 6: 2999.5, 7: 7499.5, 8: 14999.5,
                               9: 20000},
                        var='V44'))])

In [13]:
y_train = x['V86'].iloc[1160:]
y_test = x['V86'].iloc[0:1160]
del x['V86']
X_train = x.iloc[1160:]
X_test = x.iloc[0:1160]

In [15]:
x_train = pipe.transform(X_train)
x_test = pipe.transform(X_test)

In [21]:
# params={'class_weight':[None],
#         'penalty':['l1','l2'],
#         'C':[.0001,.0005,.001,.005,.01,.05,.1,1,2,5]}

# model=LogisticRegression(fit_intercept=True)

# grid_search=GridSearchCV(model,
#                          param_grid=params,
#                          cv=10,
#                          scoring="roc_auc",
#                          n_jobs=-1,
#                          verbose=20)

clf = RandomForestClassifier()
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None],
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

n_iter_search = 10
# n_iter parameter of RandomizedSeacrhCV controls, how many 
# parameter combination will be tried; out of all possible given values

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   scoring='roc_auc',
                                   cv=10,
                                  n_jobs=-1,verbose=20)

In [23]:
random_search.fit(x_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': [None],
                                        'criterion': ['entropy', 'gini'],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'max_features': [5, 10, 20, 25, 30, 35],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300, 500,
                                                         700, 1000]},
                   scoring='roc_auc', verbose=20)

In [34]:
# grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1,
                               2, 5],
                         'class_weight': [None], 'penalty': ['l1', 'l2']},
             scoring='roc_auc', verbose=20)

In [35]:
grid_search.best_estimator_

LogisticRegression(C=0.05)

In [24]:
report(random_search.cv_results_, 3)

Model with rank: 1
Mean validation score: 0.746284 (std: 0.054231)
Parameters: {'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 5, 'max_depth': 5, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': False}

Model with rank: 2
Mean validation score: 0.744737 (std: 0.044458)
Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 25, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': False}

Model with rank: 3
Mean validation score: 0.741485 (std: 0.050523)
Parameters: {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 15, 'max_features': 20, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}



In [51]:
logr = RandomForestClassifier(**{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 15, 
                                 'max_features': 20, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True})

In [53]:
logr.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=30, max_features=20,
                       min_samples_leaf=15, min_samples_split=10,
                       n_estimators=300)

In [125]:
train_score=logr.predict_proba(x_test[imp_feat])[:,1]
real=y_test

cutoffs=np.linspace(0.01,0.99,99)

In [127]:
f_beta_score={}

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP

    precision = (TP/TP+FP)*(TP/TP+FN)
    recall = (TP/TP+FP)+(TP/TP+FN)
    
      
    f_beta=((1+4)*precision)/recall
       
    f_beta_score[cutoff] = f_beta

In [129]:
for i in f_beta_score.keys():
    if f_beta_score[i] == max(f_beta_score.values()):
        print(i, f_beta_score[i])

0.13 174.3131868131868


In [137]:
test = pipe.transform(df_test)

In [155]:
predicted = (logr.predict_proba(test[imp_feat])[:,1]>0.15).astype(int)

In [159]:
pd.DataFrame(predicted).to_csv('submission.csv', index=False)

In [157]:
predicted.sum()

402

In [172]:
(170/test.shape[0])*100

4.25

In [174]:
f_beta_score

{0.01: 4.994934143870314,
 0.02: 19.897828863346106,
 0.03: 39.5,
 0.04: 63.405660377358494,
 0.05: 81.68577981651376,
 0.060000000000000005: 94.76439790575917,
 0.06999999999999999: 111.27272727272727,
 0.08: 130.88926174496643,
 0.09: 152.64598540145985,
 0.09999999999999999: 174.57831325301206,
 0.11: 178.34101382488478,
 0.12: 181.22448979591837,
 0.13: 176.4,
 0.14: 170.8860759493671,
 0.15000000000000002: 163.79310344827587,
 0.16: 154.1860465116279,
 0.17: 145.42372881355934,
 0.18000000000000002: 138.46846846846847,
 0.19: 132.31132075471697,
 0.2: 124.15841584158416,
 0.21000000000000002: 119.1919191919192,
 0.22: 110.52631578947368,
 0.23: 100.54945054945055,
 0.24000000000000002: 75.30864197530865,
 0.25: 60.57692307692308,
 0.26: 50.76923076923077,
 0.27: 43.421052631578945,
 0.28: 35.733333333333334,
 0.29000000000000004: 31.68918918918919,
 0.3: 31.733333333333334,
 0.31: 31.733333333333334,
 0.32: 27.6,
 0.33: 27.63157894736842,
 0.34: 23.333333333333332,
 0.350000000000

In [178]:
15*12

180

In [180]:
1.8+1.8

3.6

In [182]:
5.4-3.6

1.8000000000000003

In [198]:
(x['V86'].sum()/len(x))*100

5.977327378907591

In [194]:
len(x)

5822

In [69]:
feat_imp_df=pd.DataFrame({'features':x_train.columns,
                          'importance':logr.feature_importances_})

feat_imp_df=feat_imp_df.sort_values('importance',ascending=False)
feat_imp_df['normalised_imp']=feat_imp_df['importance']/np.sum(feat_imp_df['importance'])
feat_imp_df['cum_imp']=np.cumsum(feat_imp_df['normalised_imp'])

imp_feat = list(feat_imp_df.head(50)['features'])

imp_feat_df = pd.concat([x_train, x_test])[imp_feat]

In [123]:
logr.fit(x_train[imp_feat], y_train)

RandomForestClassifier(criterion='entropy', max_depth=30, max_features=20,
                       min_samples_leaf=15, min_samples_split=10,
                       n_estimators=300)